In [ ]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

In [ ]:
# 학습 데이터(MNIST)
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/", one_hot=True)

In [ ]:
# X(이미지)와 Y(숫자값)의 입력값
X = tf.placeholder(tf.float32, [None, 28*28]) # 무한대 x 784(28*28) 행렬
Y = tf.placeholder(tf.int64, [None]) # 무한대 x 1 행렬

In [ ]:
# CNN Model
def model(input_X):
    x_reshape = tf.reshape(X, [-1, 28, 28, 1])

    # Conv 레이어1
    W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 20], stddev=0.1)) # Filter가 Weight 역활을 함
    b_conv1 = tf.Variable(tf.zeros([20]))
    h_conv1 = tf.nn.conv2d(x_reshape, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1
    fmap_conv1 = tf.nn.tanh(h_conv1) # -> Feature(Activation) Map 생성

    # Pooling(Max) 레이어1
    h_pool1 = tf.nn.max_pool(fmap_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # fully-connected 레이어 1
    W_fc1 = tf.Variable(tf.truncated_normal([20 * 14 * 14, 500], stddev=0.1))
    b_fc1 = tf.Variable(tf.zeros([500]))
    h_pool1_flat = tf.reshape(h_pool1, [-1, 20 * 14 * 14])
    h_fc1 = tf.nn.tanh(tf.matmul(h_pool1_flat, W_fc1) + b_fc1)

    # 출력(fully connected) 레이어 2 (10개 출력)
    class_num = 10
    W_fc2 = tf.Variable(tf.truncated_normal([500, class_num], stddev=0.1))
    b_fc2 = tf.Variable(tf.zeros([class_num]))
    pred = tf.matmul(h_fc1, W_fc2) + b_fc2

    return pred

pred = model(X)

In [ ]:
# Cost Function(Cross Entropy)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=pred)
cost = tf.reduce_mean(cross_entropy)

# Gradient descent Optimizer(학습)
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [ ]:
# 학습 시작
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(10000):
    X_images, X_labels = mnist.train.next_batch(batch_size=100)
    X_labels = np.where(X_labels)[1]
#     print(X_labels)
        
    sess.run(optimizer, feed_dict={X: X_images, Y: X_labels})

    # 로그
    training_cost = sess.run(cost, feed_dict={X: X_images, Y: X_labels})
    print(epoch, training_cost)

print("학습완료! (cost : " + str(training_cost) + ")")

In [ ]:
test_images, test_labels = mnist.test.next_batch(16)
test_labels = np.where(test_labels)[1]
    
batch_size = 100
nCnt = 0
for i in range(1):
    x_test, test_label = mnist.test.next_batch(batch_size)
    arr_data = sess.run(pred, feed_dict={X: x_test})

    pred_val = tf.argmax(arr_data, 1)
    real_val = np.where(test_label)[1]
    
    for ib in range(batch_size):
        tmp_pred = pred_val.eval(session=sess)
        tmp_real = real_val
        
        
        if tmp_pred[ib] == tmp_real[ib]:
            print("예측값: " + str(tmp_pred[ib]) + " / 실제값: " + str(tmp_real[ib]) + " => True")
            nCnt += 1
        else:
            print("예측값: " + str(tmp_pred[ib]) + " / 실제값: " + str(tmp_real[ib]) + " => False")

print("Accuracy:", (nCnt/100.0)*100)